In [ ]:
pip install networkx matplotlib

In [48]:
# Setup
from IPython.display import display, Markdown, Latex, Math
from itertools import combinations
import math
import random
import heapq
import matplotlib.pyplot as plt
import networkx as nx

# ISBN 10

## Prüfziffer berechnen

In [2]:
def calculateCheckDigit(isbn10):
    sum = 0
    for i in range(9):
        sum += (i + 1) * int(isbn10[i])
    checkDigit = sum % 11
    if checkDigit == 10:
        return 'X'
    else:
        return str(checkDigit)

In [ ]:
isbn = "305501517"
display(Markdown(f"isbn10: {isbn}-**{calculateCheckDigit(isbn)}**"))

## Zahlendreher erkennen
![Zahlendreher](images/isbn10_zahlendreher.png)

# Inverse/Nullteiler/Ordnung  erkennen
[a] ∈ ℤ_m besitzt genau dann ein Inverses [a]^–1, wenn ggT( a, m ) = 1.

In [4]:
def Inverses(a, m):
    for i in range(m):
        if (a * i) % m == 1:
            return i
    return None

In [ ]:
Inverses(6, 11)
# 6 * 2 = 12 % 11 = 1

Andernfalls ist [a] ein Nullteiler, d.h. es gibt ein [b] ∈ ℤm mit [a⋅b] = [0].

In [6]:
def NullTeilerErkennen(n):
    nullteiler = []
    for i in range(2, n):
        if n % i == 0:
            nullteiler.append(i)
    return nullteiler

In [ ]:
NullTeilerErkennen(4)
# 2 * 2 = 4 % 4 = 0

*Ordnung*: Die Funktion order(a, p) berechnet die Ordnung eines Elements a in Z^∗_p​, indem sie die kleinste positive Ganzzahl k findet, für die a^k≡ 1 mod p.  

*Primitive Elemente*: Primitive Elemente sind diejenigen, deren Ordnung gleich φ(17)=16 ist. Dies wird durch die Bedingung ord_value == p - 1 erreicht.

In [8]:
def order(a, p):
    k = 1
    current = a % p
    e = f"{a}^{k}={current}%{p}, "
    while current != 1:
        current = (current * a) % p
        k += 1
        e += f"{a}^{k}={current}%{p}, "
    print(e[:-2])
    return k

In [ ]:
# Gegebene Zahlen
element = 3
p = 17

ord = order(element, p)
print(f"ord({element})={ord}")

if ord == p - 1:
    print(f"Element {element} ist ein Erzeuger von Z_{p}, primitives Element")
else:
    print(f"Element {element} ist kein Erzeuger von Z_{p}")

#  System voller Kongruenzen mit z===a mod k, mit z=const.

In [10]:
def solve_congruence(z, a, k):
    if (z - a) % k == 0:
        return True, (z - a) // k
    else:
        return False, None

In [ ]:
# Beispielwerte
z = 10  # Konstante
a = 2   # Wert der Kongruenz
k = 4   # Modulus

ergebnis, vielfaches = solve_congruence(z, a, k)
if ergebnis:
    print(f"{z} ≡ {a} (mod {k}) ist erfüllt. Vielfaches: {vielfaches}")
else:
    print(f"{z} ≡ {a} (mod {k}) ist nicht erfüllt.")

# GGT, GGT Rückwärts, Lemma von Bezout

## GGT - Euklidischer Algorithmus

In [12]:
def ggT(a, b, verbose=True):
    while b != 0:
        reminder = a % b
        factor = a // b
        if verbose:
            print(f"{a} = {factor} * {b} + {reminder}")
        a = b
        b = reminder
    return a

In [ ]:
a = 2406
b = 654
display(Markdown(f"ggt({a}, {b}) = **{ggT(a, b)}**"))

## Euklidischer Algorithmus rückwärts

In [14]:
def ggTRueckwaerts(a, b):
    if a == 0:
        return b, 0, 1
    gcd, x1, y1 = ggTRueckwaerts(b % a, a)
    x = y1 - (b // a) * x1
    y = x1
    return gcd, x, y

In [ ]:
a = 2406
b = 654
gcd, x, y = ggTRueckwaerts(a, b)
display(Markdown(f"{gcd} = **{x}** * {a} + **{y}** * {b}"))

## Lemma von Bezout
![Zahlendreher](images/lemma_bezout.png)

In [16]:
def lemma_bezout(a, b):
    print(f"{a} * x - {b} * y = 0\n")
    print("=== ggt ===")
    gcd = ggT(a, b)
    print(f"\nggt({a}, {b}) = {gcd}")
    print("=== ggt ===\n")

    # allgemeine Lösung xn = (x y)^t
    x = b // gcd
    y = a // gcd
    r = a * x - b * y


    display(Math(rf"x = \begin{{pmatrix}} x \\ y \end{{pmatrix}} = x_p + x_n t"))

    print(f"allgemeine Lösung:")

    display(Math(rf'2406 \cdot \frac{{{b}}}{{{gcd}}} - 654 \cdot \frac{{{a}}}{{{gcd}}} = {a} \cdot {x} - {b} \cdot {y} = {r}'))
    allgemeineLoesung = rf"\begin{{pmatrix}} {x} \\ {y} \end{{pmatrix}}"
    display(Math(rf'x_n = {allgemeineLoesung}'))

    # partikuläre Lösung p = (px py)^t
    _, px, py, = ggTRueckwaerts(a, b)
    partikiulaereLoesung = rf"\begin{{pmatrix}} {px} \\ {py} \end{{pmatrix}}"
    print(f"partikuläre Lösung:")
    display(Math(rf'x_p = {partikiulaereLoesung}'))

    print("alle Lösungen:")
    display(Math(rf'x = \begin{{pmatrix}} x \\ y \end{{pmatrix}} = {partikiulaereLoesung} + {allgemeineLoesung}t'))
    display(Math(rf"{a} ({px} + {x}t) - {b} ({py} + {y}t) = {gcd}"))

In [ ]:
a = 2406
b = 654
lemma_bezout(a, b)

# Chinesischer Restsatz - simultane Kongruenzen

In [18]:
def chinesischer_restklassensatz(congruences):
    print("Chinesischer Restklassensatz:")
    # Extract divisors and remainders from tuples
    num = [n for n, r in congruences]
    rem = [r for n, r in congruences]

    # Compute product of all numbers
    prod = 1
    for n in num:
        prod *= n
    print(f"mod = {" * ".join(map(str, num))} = {prod}\n")

    terms = []
    result = 0
    # Apply CRT formula
    for n, r in zip(num, rem):
        pp = prod // n
        gcd, inv, _ = ggTRueckwaerts(pp, n)
        inv = inv % n  # Ensure the inverse is positive
        term = r * pp * inv
        result += term
        terms.append(term)
        print(f"Calculating term for modulus {n}:")
        print(f"pp = {pp}, inverse = {inv}")
        print(f"Term = {r} * {pp} * {inv} = {term}\n")

    result = result % prod
    print(f"Final result:")
    print(f"z = {" + ".join(map(str, terms))} = {result} mod {prod}")
    return result

In [ ]:
# z = 3 (mod 5)
# z = 4 (mod 7)
# z = 5x + 3 = 7y + 4
congruences = [(5, 3), # mod 5, rest 3
               (7, 4) # mod 7, rest 4
               ]
chinesischer_restklassensatz(congruences)

In [ ]:
# z = 3 (mod 11) 
# z = 6 (mod 8)
# z = 1 (mod 15)
congruences = [
    (11, 3), # mod 11, rest 3
    (8, 6), # mod 8, rest 6
    (15, 1) # mod 15, rest 1
    ]
chinesischer_restklassensatz(congruences)

# Primfaktorzerlegung

In [21]:
def p_factorization(n):
    i = 2
    lst = []
    while i * i <= n:
        if n % i:
            i += 1
        else:
            n //= i
            lst.append(i)
    if n > 1:
        lst.append(n)
    return lst

In [ ]:
p_factorization(20)

## Euler'sche Phi-Funktion
![Euler'sche Phi-Funktion](images/euler_phi.png)

In [23]:
def eulerPhi(n):
    factors = p_factorization(n)
    phi = n
    for factor in factors:
        phi *= 1 - 1/factor
    return int(phi)

In [ ]:
eulerPhi(20)

# Binomialkoeffizienten

![binomialkoeffizienten](./images/binomialkoeffizienten.png)

In [25]:
def binomial_coefficient(n, k):
    if k > n or k < 0:
        return 0
    return math.comb(n, k)

In [ ]:
# Beispielwerte
n = 5  # Gesamtanzahl der Objekte
k = 3  # Anzahl der zu ziehenden Objekte

ergebnis = binomial_coefficient(n, k)
print(f"Die Anzahl der Möglichkeiten, {k} Objekte aus {n} zu ziehen, ist: {ergebnis}")

![binomische_lehrsatz.png](./images/binomische_lehrsatz.png)


In [27]:
def binomial_expansion(a, b, n):
    expansion = []
    for k in range(n + 1):
        coefficient = math.comb(n, k)
        term = f"{coefficient}*{a}^{n-k}*{b}^{k}"
        expansion.append(term)
    return " + ".join(expansion)

In [ ]:
# Beispielwerte
a = 'x'  # erster Term
b = 'y'  # zweiter Term
n = 3    # exponent

ergebnis = binomial_expansion(a, b, n)
print(f"Die Expansion von ({a} + {b})^{n} ist: {ergebnis}")

# Tunierplan - Rundenturnier - Blockplan - BIBD
![bibd](images/bibd.png)
  

Parameter:
- v: Anzahl der Varietäten
- k: Länge eines Blocks
- λ (lambda): sagt, wie oft jede 2-elementige Teilmenge der Varietäten {1, 2, …, v} im Blockplan vorkommt
- b: Anzahl der Blöcke (b = v über 2)
- r: Wie oft jede Varietät in einem Block vorkommt (r = v - 1)

In [29]:
def Tunierplan(v):
    pairings = []

    for round in range(1, v):
        round_pairings = []

        for x in range(1, v):
            for y in range(x + 1, v):

                if (x + y) % (v - 1) == round % (v - 1):
                    round_pairings.append((x, y))

            if (x + x) % (v - 1) == round % (v - 1):
                round_pairings.append((x, v))

        pairings.append(round_pairings)
    return pairings

In [ ]:
v = teams = 8
plan = Tunierplan(v)

table_header = f"|{'|'.join([f" Runde {i} " for i in range(1, len(plan)+ 1)])} |\n"
table_sep = f"|{'|'.join(["----------" for i in range(1, len(plan)+ 1)])}|\n"

table_body = ""
for row in range(len(plan[0])):
    for col in range(len(plan)):
        table_body += f"|{plan[col][row]}"
    table_body += "|\n"

display(Markdown(table_header + table_sep + table_body))

# Prinzip der Inklusion und Exklusion – Siebmethode

In [31]:
def calculate_others(total, counts):

    keys = [key for key in counts.keys() if len(key) == 1]
    characteristics = len(counts)
    with_characteristic = 0

    display(Math(f"|\\Omega| = 73 \\; |r| = {len(keys)}"))
    latex = "|\\Omega|"

    for r in range(1, characteristics + 1):
        latex += "+" if r % 2 == 0 else "-"
        latex += "("        
        for combo in combinations(range(characteristics), r):
            count_key = ''.join(chr(65 + i) for i in combo)

            if count_key in counts:
                latex += f"|{count_key}| + " if len(count_key) == 1 else f"|{' \\cap '.join(count_key)}| + "
                with_characteristic += (-1) ** (r + 1) * counts[count_key]

        latex = latex[:-2] + ")"
    
    while '))' in latex:
        latex = latex.replace('))', ')')
    latex += f" = {total - with_characteristic}"
    display(Math(latex))
    
    return total - with_characteristic

In [ ]:
total = 55 # 55 Athleten
counts = {
    'A': 35, # Fußball
    'B': 27, # Leichtathletik
    'C': 12, # Judo
    # 'D': 1, # Handball
    'AB': 13, # Fußball, Leichtathletik
    'AC': 7, # Fußball, Judo
    'BC': 5, # Leichtathletik, Judo
    'ABC': 2, # Fußball, Leichtathletik, Judo
    # 'ABCD': 1 # Fußball, Leichtathletik, Judo, Handball
}

print(f"Andere: {calculate_others(total, counts)}")

In [ ]:
total = 73
counts = {
    'A': 20,    
    'B': 25,    
    'C': 52,    
    'AB': 7,     
    'AC': 12,    
    'BC': 17,    
    'ABC': 1
}

print(f"Andere: {calculate_others(total, counts)}")

# RSA - Verschlüsselung

## Prime

In [34]:
def is_prime(n):
  # https://stackoverflow.com/questions/15285534/isprime-function-for-python-language
  if n == 2 or n == 3: return True
  if n < 2 or n%2 == 0: return False
  if n < 9: return True
  if n%3 == 0: return False
  r = int(n**0.5)
  # since all primes > 3 are of the form 6n ± 1
  # start with f=5 (which is prime)
  # and test f, f+2 for being prime
  # then loop by 6. 
  f = 5
  while f <= r:
    # print('\t',f)
    if n % f == 0: return False
    if n % (f+2) == 0: return False
    f += 6
  return True   

In [35]:
def generate_prime_candidate(length):
    p = 0
    while not is_prime(p):
        p = random.getrandbits(length)
    return p

In [36]:
def generate_keys(bit_length=8):
    # Zwei zufällige Primzahlen wählen (für mehr Sicherheit größere Bitlängen verwenden)
    p = generate_prime_candidate(bit_length)
    q = generate_prime_candidate(bit_length)
    while p == q:  # Sicherstellen, dass p und q unterschiedlich sind
        q = generate_prime_candidate(bit_length)

    n = p * q
    phi = (p - 1) * (q - 1)

    # Wählen Sie e, 1 < e < phi und e ist relativ prim zu phi
    e = random.randrange(1, phi)
    while ggT(e, phi, False) != 1:
        e = random.randrange(1, phi)

    # Berechnen des modularen Inversen von e
    d = Inverses(e, phi)

    # Öffentlichen und privaten Schlüssel zurückgeben
    return (e, n), (d, n), p, q, phi

## RSA

In [ ]:
bit_length = 8
(e, n), (d, m), p, q, phi = generate_keys(bit_length)
print(f"Öffentlicher Schlüssel: ({e}, {n})")
print(f"Privater Schlüssel: ({d}, {m})")
print(f"p: {p}")
print(f"q: {q}")
print(f"phi: {phi}")

In [ ]:
# Änderbare Parameter
# p, q, e, d, w

p = 11 # Primzahl, groß
q = 13 # Primzahl, groß
m = p * q # Modulus, ist öffentlich, schwer zu faktorisieren
print(f"p, q gültig: {is_prime(p) and is_prime(q)}")

# e und phi müssen teilerfremd sein
e = 13 # öffentlicher Schlüssel
phi = eulerPhi(m)
print(f"e und phi sind teilerfremd: {ggT(e, phi, False) == 1}")

d = 77 # privater Schlüssel
w = 8 # Nachricht

c = w ** e % m # Verschlüsselung
w_1 = c ** d % m # Entschlüsselung

print(f"c = {w}^{e} mod {m} = {c}")
print(f"w_1 = {c}^{d} mod {m} = {w_1}")

# Perfekte Zahlen

Eine perfekte Zahl ist eine positive ganze Zahl n, die gleich der Summe ihrer positiven Teiler (außer sich selbst) ist. Zum Beispiel ist 6 eine perfekte Zahl, weil ihre Teiler 1, 2 und 3 sind und 1+2+3=6.
Um zu zeigen, dass nn eine perfekte Zahl ist, musst du folgende Schritte durchführen:
- Finde die positiven Teiler von nn (außer nn selbst).
- Berechne die Summe dieser Teiler.
- Zeige, dass diese Summe gleich n ist.
- 
σ(n): Summe der positiven Teiler von n.
d(n): Anzahl der positiven Teiler von nn.

In [39]:
def is_perfect_number(n):
    if n <= 1:
        return False
    divisors = list(i for i in range(1, n) if n % i == 0)
    divisors_sum = sum(divisors)
    return (divisors_sum == n), divisors

In [ ]:
# Beispielwert
n = 6  # Prüfe, ob 6 eine perfekte Zahl ist
b, divisors = is_perfect_number(n)
if b:
    print(f"{n} ist eine perfekte Zahl. Teiler: {divisors}, σ({n}) = {sum(divisors)}")
else:
    print(f"{n} ist keine perfekte Zahl. Teiler: {divisors}, Summe: {sum(divisors)}")

print(f"d({n}) = {len(divisors)}")

# Berechnung der Determinante einer Matrix

In [41]:
def determinant(matrix):
    # Basisfall für 1x1-Matrix
    if len(matrix) == 1:
        return matrix[0][0], f"{matrix[0][0]}"
    
    # Basisfall für 2x2-Matrix
    if len(matrix) == 2:
        formel = f"{matrix[0][0]} * {matrix[1][1]} - {matrix[0][1]} * {matrix[1][0]}"
        return matrix[0][0] * matrix[1][1] - matrix[0][1] * matrix[1][0], formel
    
    det = 0
    formeln = []
    for c in range(len(matrix)):
        # Erzeugt eine Untermatrix durch Entfernen der ersten Zeile und der c-ten Spalte
        submatrix = [row[:c] + row[c+1:] for row in matrix[1:]]
        # Rekursive Berechnung der Determinante
        d, sub_formel = determinant(submatrix)
        det += ((-1) ** c) * matrix[0][c] * d

        formel = f"{"+ " if ((-1) ** c) == 1 else "- "} {matrix[0][c]} * ({sub_formel})"
        formeln.append(formel)
    
    return det, f"{' '.join(formeln)}"[2:]

In [ ]:
# Definition der Matrix L
matrix = [
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12],
    [13, 14, 15, 16]
]

det, f = determinant(matrix)
print(f"Die Determinante der Matrix L ist: {det}")
print(f"Die Formel für die Determinante lautet: {f}")


# Dijkstra-Algorithmus

In [54]:
def dijkstra(graph, start):
    # Initialisiere die Distanzen mit Unendlichkeit und setze die Startdistanz auf 0
    distances = {vertex: float('infinity') for vertex in graph}
    distances[start] = 0

    predecessors = {vertex: None for vertex in graph}

    # Verwende einen Vorrangwarteschlange für die zu verarbeitenden Knoten
    priority_queue = [(0, start)]
    
    # Solange die Warteschlange nicht leer ist
    while priority_queue:
        # Wähle den Knoten mit der kleinsten Distanz
        current_distance, current_vertex = heapq.heappop(priority_queue)

        # Wenn die aktuelle Distanz größer ist als die bekannte, überspringe diesen Knoten
        if current_distance > distances[current_vertex]:
            continue

        # Überprüfe die Nachbarn des aktuellen Knotens
        for neighbor, weight in graph[current_vertex].items():
            distance = current_distance + weight

            # Wenn ein kürzerer Pfad gefunden wird, aktualisiere die Distanz und füge zur Warteschlange hinzu
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                predecessors[neighbor] = current_vertex
                heapq.heappush(priority_queue, (distance, neighbor))

    # Konstruiere die Pfade von Start zu allen anderen Knoten
    paths = {}
    for vertex in graph:
        path = []
        step = vertex
        while step is not None:
            path.insert(0, step)
            step = predecessors[step]
        if path[0] == start:  # Prüfe, ob ein gültiger Pfad existiert
            paths[vertex] = path

    return distances, paths

In [53]:
def draw_graph(graph):
    G = nx.Graph()

    # Füge Kanten mit Gewichten hinzu
    for node, edges in graph.items():
        for neighbor, weight in edges.items():
            G.add_edge(node, neighbor, weight=weight)
    
    # Zeichne den Graphen
    pos = nx.spring_layout(G)  # Positionierung der Knoten
    nx.draw(G, pos, with_labels=True, node_color='lightblue', node_size=2000, font_size=10, font_weight='bold')
    labels = nx.get_edge_attributes(G, 'weight')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)
    
    # Zeige den Graphen an
    plt.title('Graphische Darstellung des Graphen')
    plt.show()

In [ ]:
graph = {
    'A': {'B': 1, 'C': 4},
    'B': {'A': 1, 'C': 2, 'D': 5},
    'C': {'A': 4, 'B': 2, 'D': 1},
    'D': {'B': 5, 'C': 1}
}

# Finde den kürzesten Pfad von Knoten 'A'
distances, paths = dijkstra(graph, 'A')
print(distances)
print("Kürzeste Pfade:")
for destination, path in paths.items():
    print(f"{destination}: {' -> '.join(path)}")
draw_graph(graph)